In [25]:
# https://github.com/chris-lovejoy/job-scraper/blob/master/job_scraper.py
import urllib
import requests
from bs4 import BeautifulSoup
import selenium
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
import pandas as pd
import os

import time



In [40]:
job_title = "Data Analyst"
location = "London"

# Loop over several pages until there no results returned.
# indeed returns 10 results per page

getVars = {'q' : job_title, 'l' : location, 'fromage' : 'last', 'sort' : 'date', 'start' : 0}
url = ('https://www.indeed.co.uk/jobs?' + urllib.parse.urlencode(getVars))
page = requests.get(url)
soup = BeautifulSoup(page.content, "html.parser")
job_soup = soup.find(id="resultsCol")


In [51]:
# Functions to extract parts of the job
def extract_title_indeed(job_elem):
    title_elem = job_elem.find('h2', class_='title')
    title = title_elem.text.strip()
    return title

def extract_company_indeed(job_elem):
    company_elem = job_elem.find('span', class_='company')
    company = company_elem.text.strip()
    return(company)

def extract_link_indeed(job_elem):
    link_elem = job_elem.find('a')['href']
    link = link_elem
    return link

def extract_date_indeed(job_elem):
    date_elem = job_elem.find('span', class_='date')
    date = date_elem.text.strip()
    return(date)



# ToDo: ID // Description // Agency?
# Check this one for job description: https://towardsdatascience.com/web-scraping-job-postings-from-indeed-com-using-selenium-5ae58d155daf




def get_indeed(indeed_country_url, job_title, location, n_page_max):

    # Possible elements of job description -> do we need this?
    elem_select = ['titles', 'companies', 'links', 'date_listed']

    # Initialize collection dict
    results = {}

    # Start with page 1
    n_page = 1

    # Loop over possible page results
    while n_page <= n_page_max:

        getVars = {'q' : job_title, 'l' : location, 'fromage' : 'last', 'sort' : 'date', 'start' : (n_page - 1) * 10}
        url = (indeed_country_url + urllib.parse.urlencode(getVars))
        page = requests.get(url)
        soup = BeautifulSoup(page.content, "html.parser")
        job_soup = soup.find(id = "resultsCol")

        # Get all elements 
        job_elems = job_soup.find_all('div', class_='jobsearch-SerpJobCard')
        
        # Initialize collectors
        titles = []
        companies = []
        links = []
        date_listed = []

        for job_elem in job_elems:
            titles.append(extract_title_indeed(job_elem))
            companies.append(extract_company_indeed(job_elem))
            links.append(extract_link_indeed(job_elem))
            date_listed.append(extract_date_indeed(job_elem))

        results['titles'] = titles
        results['companies'] = companies
        results['link'] = links
        results['date_listed'] = date_listed

        # Follow link of each job element to extract details from the job description webpage


        # Next 
        n_page += 1

        # Sleep for 1 second
        time.sleep(1)

    return results







In [56]:
job_elems = job_soup.find_all('div', class_='jobsearch-SerpJobCard')
job_elem = job_elems[0]
job_elem

https://www.indeed.co.uk/rc/clk?jk=ca58b39fdfcd8c33&amp;fccid=5e964c4afc56b180&amp;vjs=3

<div class="jobsearch-SerpJobCard unifiedRow row result" data-jk="ca58b39fdfcd8c33" data-tn-component="organicJob" id="p_ca58b39fdfcd8c33">
<h2 class="title">
<a class="jobtitle turnstileLink" data-tn-element="jobTitle" href="/rc/clk?jk=ca58b39fdfcd8c33&amp;fccid=5e964c4afc56b180&amp;vjs=3" id="jl_ca58b39fdfcd8c33" onclick="setRefineByCookie([]); return rclk(this,jobmap[0],true,0);" onmousedown="return rclk(this,jobmap[0],0);" rel="noopener nofollow" target="_blank" title="Data and Analytics Academy">
<b>Data</b> and Analytics Academy</a>
<span class="new">new</span></h2>
<div class="sjcl">
<div>
<span class="company">
<a class="turnstileLink" data-tn-element="companyName" href="/cmp/Pwc" onmousedown="this.href = appendParamsOnce(this.href, 'from=SERP&amp;campaignid=serp-linkcompanyname&amp;fromjk=ca58b39fdfcd8c33&amp;jcid=5e964c4afc56b180')" rel="noopener" target="_blank">
PwC</a></span>
<span class="ratingsDisplay">
<a class="ratingNumber" data-tn-variant="cmplinktst2" href="/cmp/Pwc

In [32]:
titles = get_indeed(indeed_country_url = 'https://www.indeed.co.uk/jobs?', job_title = "Data", location = "London", n_page_max = 3)

In [47]:
titles = get_indeed(indeed_country_url = 'https://de.indeed.com/jobs?', job_title = "Data", location = "Munich", n_page_max = 1)

In [52]:
titles = get_indeed(indeed_country_url = 'https://indeed.com/jobs?', job_title = "Data", location = "Boston", n_page_max = 1)

In [53]:
pd.DataFrame.from_dict(titles)

,titles,companies,link,date_listed
0,Research Intern\nnew,Perceptive Automata,/rc/clk?jk=934236da4bdff168&fccid=a2b1f7c3f77a...,Just posted
1,"Senior Program Manager, Data & Analytics\nnew",ENGIE Impact,/rc/clk?jk=2623bff73fc55a04&fccid=6ae3fd9d9bc6...,Just posted
2,Enterprisewide Mgmt\nnew,"Sonos, Inc",/rc/clk?jk=fc7f1de6cc346d45&fccid=a3ad90904ce5...,Just posted
3,Ops Services Coordinator\nnew,Tatte Bakery & Cafe,/rc/clk?jk=2aa7b9b5828f8b80&fccid=b58d68165da1...,Just posted
4,Commercial Strategy and Operations Associate\nnew,Athenahealth,/rc/clk?jk=2845a28305d046d7&fccid=ca123abbc3ea...,Just posted
5,Science Writer\nnew,Massachusetts General Hospital(MGH),/rc/clk?jk=defab6446c939994&fccid=3aaabf5c54e5...,Just posted
6,Senior HR Shared Services Representative\nnew,LogMeIn,/rc/clk?jk=1f3d1b56718c6db5&fccid=b665f01429fa...,Just posted
7,Research Assistant/Clinical Training Program A...,Planned Parenthood League of Massachusetts,/rc/clk?jk=2a1d75b5163c731b&fccid=dda6e6eeb880...,Just posted
8,"VP, Responsible Investment Analyst\nnew",Amundi Asset Management,/rc/clk?jk=c621f3fc6fbadb37&fccid=25a60394650c...,Just posted
9,HR Preboarding Specialist\nnew,Brown Brothers Harriman,/rc/clk?jk=2770b6c5e0a860c8&fccid=6b4d8589813e...,Just posted


In [54]:
titles['link'][0]

'/rc/clk?jk=934236da4bdff168&fccid=a2b1f7c3f77a78d6&vjs=3'

In [15]:


job_elems = job_soup.find_all('div', class_='jobsearch-SerpJobCard')
     
cols = []
extracted_info = []

def extract_job_title_indeed(job_elem):
    title_elem = job_elem.find('h2', class_='title')
    title = title_elem.text.strip()
    return title

if 'titles' in desired_characs:
    titles = []
    cols.append('titles')
    for job_elem in job_elems:
        titles.append(extract_job_title_indeed(job_elem))
    extracted_info.append(titles)  

jobs_list = {}
    
for j in range(len(cols)):
    jobs_list[cols[j]] = extracted_info[j]

num_listings = len(extracted_info[0]) 


In [16]:
jobs_list


{'titles': ['Data and Analytics Academy\nnew',
  'Global Data - Exchanges Content Analyst\nnew',
  'Data Analyst, FIS University Program\nnew',
  'Microbiology Analyst\nnew',
  'Guidewire Data Analyst\nnew',
  'Trainee Business Intelligence Opportunity - Conference Produ...\nnew',
  'Lead Business & Data Analyst\nnew',
  'AppleCare Business Development Commercial Analyst\nnew',
  'Business Insight Analyst London\nnew',
  'Risk and Data Analyst, Credit Trading\nnew',
  'CIB Chief Data & Analytics Office - Business Management - VP\nnew',
  'Prudential Risk Management Associate/Analyst\nnew',
  'Business Information Analyst\nnew',
  'BUSINESS READINESS ANALYST\nnew',
  'Lead HR Analyst - IB Global Markets\nnew']}

In [20]:
extracted_info

[['Data and Analytics Academy\nnew',
  'Global Data - Exchanges Content Analyst\nnew',
  'Data Analyst, FIS University Program\nnew',
  'Microbiology Analyst\nnew',
  'Guidewire Data Analyst\nnew',
  'Trainee Business Intelligence Opportunity - Conference Produ...\nnew',
  'Lead Business & Data Analyst\nnew',
  'AppleCare Business Development Commercial Analyst\nnew',
  'Business Insight Analyst London\nnew',
  'Risk and Data Analyst, Credit Trading\nnew',
  'CIB Chief Data & Analytics Office - Business Management - VP\nnew',
  'Prudential Risk Management Associate/Analyst\nnew',
  'Business Information Analyst\nnew',
  'BUSINESS READINESS ANALYST\nnew',
  'Lead HR Analyst - IB Global Markets\nnew']]